## Cifrar dispositivo en linux

Cifrar todo el sistema de archivos con los archivos, un discpositivo de bloques es un dispositivo especial que sirve para almacenar cosas en cualquier unix, cualquier dispositivo de bloques es un disco duro, una USB e incluso es un archivo dentro de un  sistema de archivos que se comporta como archivo de bloques, en donde se puede tener un respaldo de todos tus archivos sin tener que comprometer la privacidad.

En los sistemas UNIX o tipo UNIX es un archivo, las particiones y los discos duros en realidad se tratan como archivos. Es decir, se tratan como archivos y dentro de ese archivo se crea un sistema de archivos.

`apt install cryptsetup -y`

Logueados como el usuario **root**, nos aseguramos de que tengamos los discos sin formato

`fdisk -l | grep sd`

***
```
root@debian12lvm1:~# fdisk -l | grep sd
Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Solaris
```
***

No hay ningun motivo por el cual tengamos que crear una particion, ya que podemos ir directamente y formatearlo.

Los pasos a realizar son cifrar el disco duro completamente, por ejemplo, como LUKS2

Por ejemplo, 

`fdisk /dev/sdb`  y usando la opcion **p** y posteriormente la opcion **q**

***
```
root@debian12lvm1:~# fdisk /dev/sdb

Bienvenido a fdisk (util-linux 2.38.1).
Los cambios solo permanecerán en la memoria, hasta que decida escribirlos.
Tenga cuidado antes de utilizar la orden de escritura.

El dispositivo no contiene una tabla de particiones reconocida.
Created a new DOS (MBR) disklabel with disk identifier 0x0e6892a3.

Orden (m para obtener ayuda): p
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Modelo de disco: VBOX HARDDISK   
Unidades: sectores de 1 * 512 = 512 bytes
Tamaño de sector (lógico/físico): 512 bytes / 512 bytes
Tamaño de E/S (mínimo/óptimo): 512 bytes / 512 bytes
Tipo de etiqueta de disco: dos
Identificador del disco: 0x0e6892a3

```
***

Ahora creamos el sistema de archivos con

`mkfs.ext4 /dev/sdb`

***
```
root@debian12lvm1:~# mkfs.ext4 /dev/sdb 
mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 1310720 4k blocks and 327680 inodes
Filesystem UUID: f7e8bd87-d887-4b42-ae5e-4635a98b0d70
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (16384 blocks): done
Writing superblocks and filesystem accounting information: done
```
***
Lo podemos montar  y trabajar con el con 

 `mount /dev/sdb /mnt/`

y comprobrando con 

`df -h`

***
```
root@debian12lvm1:~# df -h
S.ficheros     Tamaño Usados  Disp Uso% Montado en
udev             960M      0  960M   0% /dev
tmpfs            197M   864K  196M   1% /run
/dev/sda1        8.9G   1.9G  6.6G  22% /
tmpfs            984M      0  984M   0% /dev/shm
tmpfs            5.0M      0  5.0M   0% /run/lock
tmpfs            197M      0  197M   0% /run/user/1000
/dev/sdb         4.9G    24K  4.6G   1% /mnt
```
***

Para trabajar con el dispositivo hay que desmontarlo

`umount /mnt`

y comprobrando con 

`df -h`

***
```
root@debian12lvm1:~# df -h
S.ficheros     Tamaño Usados  Disp Uso% Montado en
udev             960M      0  960M   0% /dev
tmpfs            197M   868K  196M   1% /run
/dev/sda1        8.9G   1.9G  6.6G  22% /
tmpfs            984M      0  984M   0% /dev/shm
tmpfs            5.0M      0  5.0M   0% /run/lock
tmpfs            197M      0  197M   0% /run/user/1000
```
***

Entendiendo que las particiones no son mas que otro archivo dentro del archivo, lo que se realizará es crear un dispositivo de bloques que estará ligado al /dev/sdb2 y todo lo que escribamos en este archivo va a tener una especie de liga simbólica que lo va a escribir pero de forma cifrada y eso lo haremos con la siguiente comando

`cryptsetup -v --cipher aes-xts-plain64 --key-size 512 --hash sha512 --iter-time 5000 --use-random luksFormat --type luks2 /dev/sdb`

***
```
root@debian12lvm1:~# cryptsetup -v --cipher aes-xts-plain64 --key-size 512 --hash sha512 --iter-time 5000 --use-random luksFormat --type luks2 /dev/sdb
ATENCIÓN: El dispositivo /dev/sdb ya contiene una firma de superbloque 'ext4'.

¡ATENCIÓN!
==========
Esto sobreescribirá los datos en /dev/sdb de forma irrevocable.

¿Está seguro? (Teclee 'yes' en mayúsculas): YES
```
***

Debemos escribir **YES** (debe ser en mayúsculas)

Ahora nos pedirá que pongamos una clave, para fines de la práctica usaremos **uiop00**, presionamos enter y confirmamos la misma contraseña y esperamos

***
```
Introduzca la frase contraseña de /dev/sdb: 
Verifique la frase contraseña: 
La firma del superbloque 'ext4' existente en el dispositivo /dev/sdb va a ser borrada.
Ranura de claves 0 creada.
Orden ejecutada correctamente.
```
***

Sobreescribe el sistemade archivos creado anteriormente (ext4)


Ahora hay que aperturar la escritura en este nuevo dispositivo de bloques cifrado, y eso lo vamos a hacer con el siguiente comando, ademas debemos de crear el dispositivo virtual denominado **misarchivos**

`cryptsetup open --type luks2 /dev/sdb misarchivos`


***
```
root@debian12lvm1:~# cryptsetup open --type luks2 /dev/sdb misarchivos
Introduzca la frase contraseña de /dev/sdb: 
```
***

Ahora existe un nuevo dispositivo en **/dev/mapper** que se llama **misarchivos** y todo lo que escribamos alli en realidad se va a escribir en **/dev/sdb1** pero cuando llegue a **/dev/sdb1** va a estar cifrado, podemos corroborar  con los siguientes comandos como se ve el nuevo dispositivo

`ls -l /dev/mapper/`

***
```
root@debian12lvm1:~# ls -l /dev/mapper/
total 0
crw------- 1 root root 10, 236 feb  7 14:18 control
lrwxrwxrwx 1 root root       7 feb 14 14:10 misarchivos -> ../dm-0
```
***

Ahora sobre ese dispositivo cifrado podemos hacer un sistema de archivos 

`mkfs.ext4 /dev/mapper/misarchivos`

***
```
root@debian12lvm1:~# mkfs.ext4 /dev/mapper/misarchivos 
mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 1306624 4k blocks and 327040 inodes
Filesystem UUID: dfc5ca85-b6ff-4370-bf69-7a2ec16e8c2b
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (16384 blocks): done
Writing superblocks and filesystem accounting information: done 
```
***

El dispositivo **/dev/mapper/misarchivos** lo está tratando como un disco duro más, es decir, como un archivo de bloques y en todos los sistemas linux el teclado, el mouse, etc, todo es un archivo.

Ahora se realiza la prueba de funcionamiento, primero debemos montar el dispositivo

`mount /dev/mapper/misarchivos /mnt`

Y despues usamos el comando siguiente para ver el espacio disponible en los discos

`df -h`

***
```
root@debian12lvm1:~# df -h
S.ficheros              Tamaño Usados  Disp Uso% Montado en
udev                      960M      0  960M   0% /dev
tmpfs                     197M   868K  196M   1% /run
/dev/sda1                 8.9G   1.9G  6.6G  22% /
tmpfs                     984M      0  984M   0% /dev/shm
tmpfs                     5.0M      0  5.0M   0% /run/lock
tmpfs                     197M      0  197M   0% /run/user/1000
/dev/mapper/misarchivos   4.9G    24K  4.6G   1% /mnt
```
***

Y a partir de este momento, todo lo que se escriba en la carpeta donde está montado el dispositivo estará cifrado, por lo que se procede a hacer la prueba. Primero nos cambiamos al directorio **/mnt**

`cd /mnt`

Ahora lo que hacemos es crear un documento cifrado

`nano prueba.txt`

El condenido del archivo **prueba.txt** es el siguiente


***
```
Soy un texto que aparecerá cifrado
```
***

Una vez montado el dispositivo cifrado (previo conocimiento de la contraseña) el tratamiento de los archivos es totalmente transparente, si embargo, la información esta llegando al dispositivo de forma cifrada, con todo y el sistema de archivos, es decir, incluso el sistema de archivos tambien va a ser cifrado, eso incluye el nombre de los archivos y el contenido de los archivos. Esta misma configuración se le puede aplicar a un USB.

podemos ver el contenido mediante el comando **cat**

***
```
root@debian12lvm1:/mnt# cat prueba.tx 
Soy un texto que aparecerá cifrado

```
***

Una vez que hemos terminado de utilizar el disco cifrado, debemos desmontarlo

`cd ..`

`umount /mnt`

y posteriormente cerrar el cifrado.

`cryptsetup close misarchivos`

### Cifrar un archivo que se comporte como disco duro

Ya que todo es un archivo, podríamos crear un archivo, cifrar ese archivo, crearle su mapper y llevarnoslo a la nube de forma en que podríamos guardar toda nuestra información como respaldo en la nube pero de forma cifrada.

Practicamente es el mismo procedimiento, pero en lugar de usar un dispositivo de bloques, lo haremos con un archivo común y corriente.

Nos cambiaremos al directorio home del usuario con el comando

`cd`

Ahora crearemos un directrio 

`mkdir miarchivonube`

Cambiamos a ese directorio

`cd miarchivonube`

Ahora lo que tenemos que hacer es crear un archivo común y corriente, de tamaño de 512MB con el comando

`dd if=/dev/zero of=backupnube.raw bs=1048576 count=512`

***
```
root@debian12lvm1:~/miarchivonube# dd if=/dev/zero of=backupnube.raw bs=1048576 count=512
512+0 records in
512+0 records out
536870912 bytes (537 MB, 512 MiB) copied, 3.57046 s, 150 MB/s
```
***

Ahora, a ese archivo le aplicamos el mismo procedimiento con el comando que usamos para cifrar pero con el nombre del archivo **backupnube.raw**

`cryptsetup -v --cipher aes-xts-plain64 --key-size 512 --hash sha512 --iter-time 5000 --use-random luksFormat --type luks2 backupnube.raw`


***
```
root@debian12lvm1:~/miarchivonube# cryptsetup -v --cipher aes-xts-plain64 --key-size 512 --hash sha512 --iter-time 5000 --use-random luksFormat --type luks2 backupnube.raw

¡ATENCIÓN!
==========
Esto sobreescribirá los datos en backupnube.raw de forma irrevocable.

¿Está seguro? (Teclee 'yes' en mayúsculas): YES
```
***

le asignamos una contraseña, en este caso, **uiop00**

***
```
Introduzca la frase contraseña de backupnube.raw: 
Verifique la frase contraseña: 
Ranura de claves 0 creada.
Orden ejecutada correctamente.
root@debian12lvm1:~/miarchivonube# 
```
***

Ahora abrimos el archivo de la forma en la que hizo con el disco duro pero ahora con este archivo.

`cryptsetup open --type luks2 backupnube.raw backupnube`

***
```
root@debian12lvm1:~/miarchivonube# cryptsetup open --type luks2 backupnube.raw backupnube
Introduzca la frase contraseña de backupnube.raw:
root@debian12lvm1:~/miarchivonube# 
```
***

Ahora tambien existe en **/dev/mapper/** y lo comprobamos con el comando

`ls -l /dev/mapper`

***
```
root@debian12lvm1:~/miarchivonube# ls -l /dev/mapper
total 0
lrwxrwxrwx 1 root root       7 feb 14 15:12 backupnube -> ../dm-1
crw------- 1 root root 10, 236 feb  7 14:18 control
lrwxrwxrwx 1 root root       7 feb 14 14:15 misarchivos -> ../dm-0

```
***

El dispositivo de backupnube está ligado a un archivo que no es un disco duro, es un archivo común y corriente, y se le puede asignar un formato y crear un sistema de archivos.

`mkfs.ext4 /dev/mapper/backupnube`

***
```
root@debian12lvm1:~/miarchivonube# mkfs.ext4 /dev/mapper/backupnube 
mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 126976 4k blocks and 126976 inodes
Filesystem UUID: 3b880e87-c934-4699-8885-2cda81e3548e
Superblock backups stored on blocks: 
	32768, 98304

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (4096 blocks): done
Writing superblocks and filesystem accounting information: done
```
***

Y ahora lo podemos montar, en el directorio en donde nos entontramos (**/root/miarchivonube**) crearemos el directorio **backupnube** con el comando

`mkdir backupnube`

Ahora lo montamos

`mount /dev/mapper/backupnube backupnube`

Ahora lo comprobamos con 

`df -h`

***
```
root@debian12lvm1:~/miarchivonube# df -h
S.ficheros              Tamaño Usados  Disp Uso% Montado en
udev                      960M      0  960M   0% /dev
tmpfs                     197M   872K  196M   1% /run
/dev/sda1                 8.9G   2.4G  6.1G  28% /
tmpfs                     984M      0  984M   0% /dev/shm
tmpfs                     5.0M      0  5.0M   0% /run/lock
tmpfs                     197M      0  197M   0% /run/user/1000
/dev/mapper/misarchivos   4.9G    28K  4.6G   1% /mnt
/dev/mapper/backupnube    449M    24K  414M   1% /root/miarchivonube/backupnube

```
***

Si ejecutamos el siguiente comando podremos ver el contenido de la carpeta donde está montado y el archivo real en donde se guarda la información cifrada 

`ls -lh`

***
```
root@debian12lvm1:~/miarchivonube# ls -lh
total 513M
drwxr-xr-x 3 root root 4.0K feb 14 15:17 backupnube
-rw-r--r-- 1 root root 512M feb 14 15:21 backupnube.raw

```
***

y todo lo que escribamos en el directorio backupnube, en realidad se está escribiendo en este archivo.

Cuando hayamos cerrado el archivo, o reiniciado la computadora, este archivo se liberará, y todo el contenido del archivo **backupnube.raw** se encontrará cifrado, y ahora ese archivo podría subirse a la nube y quien lo descargue no lo podría abrir, ya que el cifrado que se tiene es bastante fuerte.

De esta forma, no se cifra archivo pro archivo, sino un dispositivo de bloque por completo, en donde podemos escribir de forma transparente nuestra información, con un comando lo abrimos y con un comando cerramos el dispositivo de bloques cifrado.


Para cerrar el dispositivo o archivo cerrado, primero hay que desmontarlo

`umount backupnube`

podemos verificar que se haya demontado con 

`df -h`

***
```
root@debian12lvm1:~/miarchivonube# df -h
S.ficheros              Tamaño Usados  Disp Uso% Montado en
udev                      960M      0  960M   0% /dev
tmpfs                     197M   868K  196M   1% /run
/dev/sda1                 8.9G   2.4G  6.1G  28% /
tmpfs                     984M      0  984M   0% /dev/shm
tmpfs                     5.0M      0  5.0M   0% /run/lock
tmpfs                     197M      0  197M   0% /run/user/1000
/dev/mapper/misarchivos   4.9G    28K  4.6G   1% /mnt
```
***

y posteriormente cerrar el cifrado

`cryptsetup close  backupnube`


Basado en la información compartida en [lastdragon](https://www.lastdragon.net/)




